In [1]:
import os
import numpy as np 
import pandas as pd 

In [2]:
!wget -P ./../../../dataset/ -nc https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/airquality_co_full.csv

File './../../../dataset/airquality_co_full.csv' already there; not retrieving.



In [3]:
path = "./../../../dataset/airquality_co_full.csv"
dataset_name = os.path.splitext(os.path.basename(path))[0]
data = pd.read_csv(path, index_col=0)
print("shape of data: ", data.shape)
data.head() 

shape of data:  (7344, 12)


,time_slot,day_of_week,T,RH,AH,PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),true_label,given_label,true_error
0,evening,wednesday,13.6,48.9,0.7578,1046.0,1056.0,1692.0,1268.0,0.327392,1.140560,0
1,evening,wednesday,13.3,47.7,0.7255,955.0,1174.0,1559.0,972.0,-0.090299,0.829606,0
2,evening,wednesday,11.9,54.0,0.7502,939.0,1140.0,1555.0,1074.0,0.048931,1.332621,1
3,evening,wednesday,11.0,60.0,0.7867,948.0,1092.0,1584.0,1203.0,0.048931,1.213726,1
4,night,wednesday,11.2,59.6,0.7888,836.0,1205.0,1490.0,1110.0,-0.368759,0.738148,1


In [4]:
training_data = data.drop(['true_label', 'true_error'], axis=1)
training_data.head(2)

,time_slot,day_of_week,T,RH,AH,PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),given_label
0,evening,wednesday,13.6,48.9,0.7578,1046.0,1056.0,1692.0,1268.0,1.140560
1,evening,wednesday,13.3,47.7,0.7255,955.0,1174.0,1559.0,972.0,0.829606


In [5]:
y = training_data['given_label']
X = training_data.drop(['given_label'], axis=1)

###### Add your training code here ###############
# model = 
# model_fit = model.fit(X,y)
# predictions = model.predict(X)

##### saving the predictions ####################
# savepath = "./../predictions/model_predictions.npy"
# np.save(savepath, np.array(predictions))

## Section section is relevant only if using AutoGluon

Uncomment the code in following cells and run to generate output and save to predictions folder. 

### set up arguments 

In [6]:
from autogluon.tabular import TabularPredictor

data_without_label = training_data.drop(['given_label'], axis=1)
savepath = "./../trained_models/"
if not os.path.isdir(savepath):
    os.mkdir(savepath)

predictorArgs = {
                    "path"             : savepath, 
                    "label"            : 'given_label',
                    "eval_metric"      : 'r2',
                    "problem_type"     : 'regression'
                }

hyperparameters = { 
                    'GBM'   : {}, 
                    'FASTAI': {}, 
                    'RF'    : {'criterion': 'squared_error', 
                                'ag_args': {'name_suffix': 'MSE', 
                                'problem_types': ['regression', 'quantile']}}
                    }

### Normal Model fitting

In [7]:
predictor = TabularPredictor(**predictorArgs)
predictor.fit(training_data, hyperparameters=hyperparameters)
leaderboard = predictor.leaderboard(training_data, silent=True)
leaderboard

Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    7344
Train Data Columns: 9
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1270.75 MB
	Train Data (Original)  Memory Usage: 1.35 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtyp

[1000]	valid_set's l2: 0.0463599	valid_set's r2: 0.875504


	0.9543	 = Validation score   (r2)
	5.09s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestMSE ...
	0.939	 = Validation score   (r2)
	1.44s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
	0.9586	 = Validation score   (r2)
	4.27s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.96	 = Validation score   (r2)
	0.03s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 11.15s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("./../trained_models/")
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when 

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.990140,0.954261,0.106382,0.015928,5.089764,0.106382,0.015928,5.089764,1,True,1
1,RandomForestMSE,0.987230,0.938972,0.187640,0.031033,1.436940,0.187640,0.031033,1.436940,1,True,2
2,WeightedEnsemble_L2,0.978764,0.960010,0.171649,0.027740,9.396521,0.016698,0.000172,0.032410,2,True,4
3,NeuralNetFastAI,0.967862,0.958604,0.048569,0.011640,4.274348,0.048569,0.011640,4.274348,1,True,3


### Cross Validation fitting

In [8]:
predictor_cv = TabularPredictor(**predictorArgs) 
predictor_cv.fit(training_data, presets="best_quality", num_stack_levels= 0, hyperparameters=hyperparameters)
leaderboard_cv = predictor_cv.leaderboard(training_data, silent=True)
leaderboard_cv

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    7344
Train Data Columns: 9
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1356.24 MB
	Train Data (Original)  Memory Usage: 1.35 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerat

[1000]	valid_set's l2: 0.0433034	valid_set's r2: 0.872017
[1000]	valid_set's l2: 0.0436055	valid_set's r2: 0.888051
[1000]	valid_set's l2: 0.0432074	valid_set's r2: 0.864176
[1000]	valid_set's l2: 0.0399708	valid_set's r2: 0.878169


	0.9563	 = Validation score   (r2)
	14.28s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	0.9475	 = Validation score   (r2)
	1.51s	 = Training   runtime
	0.22s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set num

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE_BAG_L1,0.992863,0.947549,0.280176,0.220799,1.507606,0.280176,0.220799,1.507606,1,True,2
1,LightGBM_BAG_L1,0.987416,0.956332,0.544128,0.087054,14.281185,0.544128,0.087054,14.281185,1,True,1
2,WeightedEnsemble_L2,0.976191,0.962578,0.879267,0.155277,40.784805,0.001558,0.000193,0.057355,2,True,4
3,NeuralNetFastAI_BAG_L1,0.969128,0.961562,0.333581,0.068030,26.446264,0.333581,0.068030,26.446264,1,True,3


### predicting and saving arrays 

In [ ]:
models = {  'normal'            : list(leaderboard['model']), 
            'crossValidation'   : list(np.append(leaderboard_cv['model'], ['oof']))}

predictors = { 'normal'          : predictor, 
               'crossValidation' : predictor_cv}

# check if path is available, else create it. 
pred_path = "./../predictions/"
if not os.path.isdir(pred_path):
    os.mkdir(pred_path)

for model_type in models.keys():
    if model_type == "normal": 
        for model in models[model_type]:
            savepath = os.path.join(pred_path, model+".npy")
            predictions = predictors[model_type].predict(data_without_label, model=model)
            np.save(savepath, np.array(predictions))
    elif model_type == "crossValidation":
        for model in models[model_type]:
            if model == "oof":
                predictions = predictors[model_type].get_oof_pred()
                savepath = os.path.join(pred_path, model+".npy")
            elif model in models['normal']:
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+"_CV.npy")
            else: 
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+".npy")

            np.save(savepath, np.array(predictions))